# <Step1. 탐색> : 데이터의 기초 정보 살펴보기 

 Chipotle 데이터셋의 기본 정보 


In [5]:
import pandas as pd

# read_csv 함수로 데이터를 DataFrame 형태로 불러온다
file_path = '../data/chipotle.tsv'
chipo = pd.read_csv(file_path, sep = '\t')

print(chipo.shape)
print("----------------------")
print(chipo.info())

ModuleNotFoundError: No module named '_bz2'

chipotle 데이터셋의 행과 열, 데이터

In [ ]:
# chipo라는 DataFrame에서 순서대로 10개의 row 데이터를 출력 
chipo.head(10).fillna(0)

In [ ]:
print(chipo.columns)
print('---------------------')
print(chipo.index)

* quantity와 item_price의 요약 통계
* order_id와 item_name의 개수

Chipotle 데이터셋의 수치적 특징 파악

* describe 함수로 요약 통계량 출력 

In [ ]:
chipo['order_id'] = chipo['order_id'].astype(str)
# order_id는 숫자의 의미를 가지지 않기 때문에 str으로 변환한다. 
print(chipo.describe())
# chipo dateframe에서 수치형 피처들의 요약 통계량을 확인

* unique 함수로 범주형 피처의 개수 출력하기

In [ ]:
print(len(chipo['order_id'].unique())) # order_id의 개수를 출력
print(len(chipo['item_name'].unique())) # item_name의 개수를 출력

# <Step2. 인사이트의 발견> : 탐색과 시각화

# 다음으로, 인사이트를 발견할 수 있을만한 개념적 질문들을 정의
* 가장 많이 주문한 item은 무엇인지
* item당 주문의 총량은 얼마인지 

# [가장 많이 주문한 item]

In [ ]:
# 가장 많이 주문한 item : top 10을 출력 
item_count = chipo['item_name'].value_counts()[:10]
for idx, (val, cnt) in enumerate(item_count.iteritems(),1):
    print("Top", idx, ":", val, cnt)

In [ ]:
chipo['item_name'].value_counts().index.tolist()[0]

# [item 당 주문 개수와 총량 구하기]

In [ ]:
# item당 주문 개수를 출력
order_count = chipo.groupby('item_name')['order_id'].count()
order_count[:10] # item당 주문 개수를 출력

In [ ]:
# item당 주문 총량을 출력
item_quantity = chipo.groupby('item_name')['quantity'].sum()
item_quantity[:10] # item당 주문 총량을 출력

# [시각화로 분석 결과 살펴보기]
* 지금까지의 분석 결과를 간단한 시각화로 표현

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

item_name_list = item_quantity.index.tolist()
x_pos = np.arange(len(item_name_list))
order_cnt = item_quantity.values.tolist()

plt.bar(x_pos, order_cnt, align = 'center')
plt.ylabel('Ordered_item_count')
plt.title('Distribution of all ordered item')

plt.show()

# 미니 퀴즈 - 1.1
* pandas에서 유용하게 사용되는 함수 value_counts()와 unique()의 차이점은 무엇인가?
    * 각 함수는 어떤 데이터 타입에 적용이 되는지, 어떤 기능을 가지고 있는지, 정확히 어떤 결과값을 반환하는지 실행해보자 

In [ ]:
print(chipo['item_name'].value_counts()[:10])

In [ ]:
print(type(chipo['item_name'].value_counts()))

In [ ]:
print(chipo['item_name'].unique()[:10])

In [ ]:
print(type(chipo['item_name'].unique()))

# <Step3. 데이터 전처리> : 나만의 조력자를 정의하자

[apply와 lambda 함수를 이용한 데이터 전처리]

In [ ]:
print(chipo.info())
print('----------------')
chipo['item_price'].head()

In [ ]:
# column 단위 데이터에 apply 함수로 전처리를 적용합니다.
chipo['item_price'] = chipo['item_price'].apply(lambda x:float(x[1:]))
chipo.describe()

In [ ]:
chipo['item_price'].head()

# <Step4. 탐색적 분석> : 스무고개로 분석하는 개념적 탐색

데이터를 이해하기 위한 조금 더 복잡한 질문들로 탐색적 데이터 분석 연습하기
* 주문당 평균 계산금액 출력하기
* 한 주문에 10달러 이상 사용한 주문의 id들 출력하기
* 각 아이템의 가격 구하기
* 가장 비싼 주문에서 item이 몇개 팔렸는지 구하기
* “Veggie Salad Bowl”이 몇 번 주문되었는지 구하기
* “Chicken Bowl”을 2개 이상 주문한 주문 횟수 구하기

# 주문당 평균 계산 금액 출력하기


In [ ]:
# 주문당 평균 계산금액을 출력 
chipo.groupby('order_id')['item_price'].sum().mean()

In [ ]:
chipo.groupby('order_id')['item_price'].sum().describe()[:10]

# [한 주문에 10달러 이상 사용한 주문 번호(id)출력하기]

In [ ]:
# 한 주문에 10달러 이상 사용한 id를 출력
chipo_orderid_group = chipo.groupby('order_id').sum()
results = chipo_orderid_group[chipo_orderid_group.item_price >= 10]

print(results[:10])
print(results.index.values)

# 각 아이템의 가격 구하기 

In [ ]:
# 각 아이템의 가격을 계산한다.
chipo_one_item = chipo[chipo.quantity == 1]
price_per_item = chipo_one_item.groupby('item_name').min()
price_per_item.sort_values(by = 'item_price', ascending = False)[:10]

In [ ]:
# 아이템 가격 분포 그래프를 출력
item_name_list = price_per_item.index.tolist()
x_pos = np.arange(len(item_name_list))
item_price = price_per_item['item_price'].tolist()

plt.bar(x_pos, item_price, align='center')
plt.ylabel('item price($)')
plt.title('Distribution of item price')

plt.show()

In [ ]:
[“Chicken Bowl”을 2개 이상 주문한 주문 횟수 구하기]
# 아이템 가격 히스토그램을 출력
plt.hist(item_price)
plt.ylabel('counts')
plt.title('Histogram of item price')

plt.show()

# 가장 비싼 주문에서 item이 총 몇개 팔렸는지 구하기

In [ ]:
# 가장 비싼 주문에서 item이 총 몇개 팔렸는지 계산
chipo.groupby('order_id').sum().sort_values(by='item_price', ascending=False)[:5]

# "Veggie Salad Bowl"이 몇 번 주문되었는지 구하기

In [ ]:
# "Veggie Salad Bowl"이 몇 번 주문되었는지를 계산
chipo_salad = chipo[chipo['item_name'] == 'Veggie Salad Bowl']
chipo_salad = chipo_salad.drop_duplicates(['item_name', 'order_id'])
# 한 주문 내에서 중복 집계된 item_name을 제거한다.

print(len(chipo_salad))
chipo_salad.head(5)

# [“Chicken Bowl”을 2개 이상 주문한 주문 횟수 구하기]
공지사항
* 'Chicken Bowl'을 2개 이상 주문한 주문 횟수 구하기 문제의 출제 의도 오류가 있었습니다. 원래의 의도는 'Chicken Bowl'을 2개 이상 주문한 고객들의 "Chicken Bowl" 메뉴에 대한 총 주문 수량에 대한 문제였습니다. 이슈를 제기해주신 장기식님께 감사드립니다.
* 따라서, 현재 문제의 의도대로 코드를 변경하였습니다. 기존의 출제 의도“Chicken Bowl”을 2개 이상 주문한 고객들의 "Chicken Bowl" 메뉴에 대한 총 주문 수량에 대한 코드는 아래에 추가해두었습니다.

In [ ]:
# "Chicken Bowl"을 2개 이상 주문한 주문 횟수를 구하기
chipo_chicken = chipo[chipo['item_name'] == "Chicken Bowl"]
chipo_chicken_result = chipo_chicken[chipo_chicken['quantity'] >= 2]
print(chipo_chicken_result.shape[0])

In [ ]:
# “Chicken Bowl”을 2개 이상 주문한 고객들의 "Chicken Bowl" 메뉴의 총 주문 수량을 구합니다.
chipo_chicken = chipo[chipo['item_name']== 'Chicken Bowl']
chipo_chicken_ordersum = chipo_chicken.groupby('order_id').sum()['quantity']
chipo_chicken_result = chipo_chicken_ordersum[chipo_chicken_ordersum >=2]

print(len(chipo_chicken_result))
chipo_chicken_result.head(5)